In [17]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [18]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

################################################# SWAP STRUCTURE BUILDERS ################################################

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'


def swap_structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end).fillna(method ='bfill')  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

################################################# BOND STRUCTURE BUILDERS ################################################
def bond_ticker(bond_name, bond_tenor):
    cmb = {'ust':1,'dbr':2,'spgb':8,'frtr':4,'btp':5,'ukt':6}
    return f'RV000{cmb[bond_name]}P {bond_tenor.upper()} BLC Curncy'

def bond_structure(bonds,structure, start, end = 'today'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} {structure}'] = bond_structure(bond1,structure, start, end)[f'{bond1.upper()} {structure}'] - bond_structure(bond2,structure, start, end)[f'{bond2.upper()} {structure}']
        return s
    else:
        if 'y' in structure.lower(): #bond o/r
            tickers = bond_ticker(bonds.lower(), structure)
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = [f'{bonds.upper()} {structure}']
            return base
        
        elif structure.lower().count('s') == 2: #bond curve
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = base[tickers[1]] - base[tickers[0]]
            return s
        
        elif structure.lower().count('s') == 3: # bond fly
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
            return s

################################################# FUTURES STRUCTURE BUILDERS ################################################

def fut_ticker(dex, contract, name = False):
    x  = ''
    if dex.lower()[:3] == 'eur':
        x = 'ER'
    elif dex.lower() == 'usd':
        x = 'SFR'
    elif dex.lower() == 'gbp':
        x = 'SFI'
    
    if name:
        return x
    else:
        return f'{x}{contract} Comdty'   
           
    
def fut_structure(dex,structure, start, end = 'today'):
    name = fut_ticker(dex.lower(), 0,True)
    
    if structure.lower().count('s') == 2: #fut curve
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = base[tickers[0]] - base[tickers[1]]
        return s
        
    elif structure.lower().count('s') == 3: # fut fly
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = fut_ticker(dex.lower(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{name}{structure}']
        return base

################################################# ASW STRUCTURE BUILDERS ################################################

def get_asw(bond_name,tenor,start, end ='today',euro = 'estr'):
    asso_swap = {'ust':'usd','dbr':'eur','spgb':'eur','frtr':'eur','btp':'eur','ukt':'gbp'}
    
    if asso_swap[bond_name] == 'eur' and euro != 'estr':
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name]+'_6',tenor)]
    else:
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name],tenor)]
    base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
    base.columns = tickers
    s = pd.DataFrame({})
    s[f'{tenor.upper()} {bond_name.upper()} ASW'] = base[tickers[1]] - base[tickers[0]]
    return s

def asw_structure(bonds,structure, start, end = 'today',euro = 'estr'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} ASW {structure}'] = asw_structure(bond1,structure, start, end,euro)[f'{bond1.upper()} ASW {structure}'] - asw_structure(bond2,structure, start, end,euro)[f'{bond2.upper()} ASW {structure}']
        return s
    else:
        if 'y' in structure.lower(): #asw o/r
            return get_asw(bonds,structure,start, end,euro)
        
        elif structure.lower().count('s') == 2: #asw curve
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = df.iloc[:,1]  - df.iloc[:,0]  
            return s
        
        elif structure.lower().count('s') == 3: # asw fly
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, belly ,long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            short_leg = short_leg.join(belly, how = 'inner')
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = (2 * df.iloc[:,1])  - (df.iloc[:,0] + df.iloc[:,2])  
            return s
        

######################################### TICKER HANDLERS #################################################################

def get_other(ticker,attribute,title,start,end='today' ):
    if 'price' in attribute.lower():
        attribute = 'px_last'
    else:
        attribute = 'YLD_YTM_MID'
    base = blp.bdh(ticker,attribute,start,end)
    base.columns = [title]
    return base

######################################### INVOICE SPREAD STRUCTURE BUILDERS ################################################


def get_ivsp(country,start, end ='today',which = 'TUA',euro ='estr'):
    tkr = {'USD':['TUAISPS','3YAISPS','FVAISPS','TYAISPS','UXYAISPS','USAISPS','WNAISPS'],
           'GER':['DUAISP','OEAISP','RXAISP','UBAISP'],
           'ITA':['BTAISP','IKAISP'],
           'FRA':['OATAISP'],
           'GBP':['GAISPO']}
    
    if country.upper() == 'USD' or country.upper() == 'GER' or country.upper() == 'ITA' :
        for i in tkr[country.upper()]:
            if which in i:
                future = i
    else:
        future = tkr[country.upper()][0]
    
    if country.lower() in ['ger','ita','fra'] and euro == 'estr':
        future += 'E'
    final_ticker = f'{future} Comdty'
    base = blp.bdh(final_ticker,'px_last',start,end)
    base.columns = [f'{future} Comdty']    
    return base
                
######################################## MEETING DATES HANDLER ##################################################################
    
def md_swap_ticker(bank, meeting,m_of_m =False):
    central_bank = {'FOMC':['USSOFED',' Curncy'],
                'ECB':['EESF','A Curncy'],
                'MPC':['GPSF','A Curncy']}
    ticker = f'{central_bank[bank.upper()][0]}{meeting}{central_bank[bank.upper()][1]}' 
    if m_of_m:
        return blp.bdp(ticker, 'SW_EFF_DT').values.flatten()[0].strftime('%b')
    else:
        return ticker
    
def get_md_swap(bank,structure,start,end  = 'today'):
    if structure.lower().count('s') == 2:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != ''] 
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = base[tickers[1]] - base[tickers[0]]
        return s
        
    if structure.lower().count('s') == 3:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != '']
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])            
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = (2*base[tickers[1]] )- (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = md_swap_ticker(bank.upper(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{md_swap_ticker(bank.upper(), structure,True)}{bank.upper()}']
        return base

    
################################################# FX HANDLER ############################################################
def get_fx_pair(pair,start,end='today'):
    ticker = f'{pair.upper()} Curncy'
    base = blp.bdh(ticker,'px_last',start,end)
    base.columns = [pair.upper()]
    return base

######################################## BOND FUTURE HANDLER ##################################################################
def get_bondf_ticker(name):
    tic_dic = {'DU':'DUA Comdty','OE':'OEA Comdty','RX': 'RXA Comdty','UB': 'UBA Comdty',
                'BTS': 'BTSA Comdty','IK': 'IKA Comdty',
                'OAT': 'OATA Comdty',
                'G':'G A Comdty',
                'TU': 'TUA Comdty','FV': 'FVA Comdty', 'TY': 'TYA Comdty','WN': 'WNA Comdty','UXY': 'UXYA Comdty'}
    return tic_dic[name.upper()]
    

def get_bond_fut(_str, start,end = 'today', fld = 'yield'):
    fld = 'YLD_YTM_MID' if fld.lower() == 'yield' else 'PX_LAST' 
    _str = _str.upper()
    label = _str.split('/')
    legs = [get_bondf_ticker(i) for i in label]
    bps = 100 if fld == 'YLD_YTM_MID' else 1
    base  = blp.bdh(legs,fld,start,end) * bps
    base.columns = label
    s = pd.DataFrame({})
    
    if _str.count('/') == 2:
        s[_str] = 2 * base[label[1]] - (base[label[0]] + base[label[2]]) 
            
    elif _str.count('/') == 1:
        s[_str] =  base[label[1]] - base[label[0]] 
                                        
    else:
        s = base
        
    return s
    

######################################## INFL SWAP HANDLER ##################################################################



######################################## MASTER HANDLER ##################################################################


def function_for_bb(trades,_from,_to):
    master_c = pd.DataFrame({})
    for i in trades.keys():
        if len(trades[i]) != 0:
            for j in trades[i]:
                if i == 'SWAP':
                    h = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                                
                elif i == 'FUTURE':
                    h = fut_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                              
                elif i == 'BOND':
                    h = bond_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'ASW':
                    h = asw_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to, euro = j[2].lower())
                elif i == 'XMKT SWAP':
                    h1 = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h2 = swap_structure(dex = j[2].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h_int = h1.join(h2, how = 'inner')
                    h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{j[0]}-{j[2]} {j[1]}'])
                elif i == 'MD SWAP':
                    h = get_md_swap(bank = j[0].upper(),structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'BOND FUTURE':
                    h = get_bond_fut(_str = j[0],start = _from , end = _to,fld =  j[1])
                elif i == 'FX':
                    h = get_fx_pair(pair = j[0].upper(),start = _from , end = _to)
                elif i == 'OTHER/CIX':
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)            

                if master_c.shape == (0,0):
                    master_c = h
                else:
                    master_c = master_c.join(h, how = 'inner')
    return master_c
                            
                            
    

In [19]:
_FROM,_TO = 2018,2023

In [20]:
def extract_auction_dates(auction_ticker,year):
    x = blp.bdh(auction_ticker,'px_last', f'{year}-01-01').index
    return x

def function_for_bb2(data_tuple,asset,_from,_to):
    if asset == 'SWAP':
        h = swap_structure(dex = data_tuple[0].lower() ,structure=data_tuple[1].lower() ,start = _from , end = _to)    
    elif asset == 'FUTURE':
        h = fut_structure(dex = data_tuple[0].lower() ,structure=data_tuple[1].lower() ,start = _from , end = _to)                              
    elif asset == 'BOND':
        h = bond_structure(bonds = data_tuple[0].lower() ,structure=data_tuple[1].lower() ,start = _from , end = _to)
    elif i ==  'BOND FUTURE':
        h = get_bond_fut(_str = j[0],start = _from , end = _to,fld =  j[1])
    elif asset ==  'ASW':
        h = asw_structure(bonds = data_tuple[0].lower() ,structure=data_tuple[1].lower() ,start = _from , end = _to, euro = data_tuple[2].lower())
    elif asset == 'XMKT SWAP':
        h1 = swap_structure(dex = data_tuple[0].lower() ,structure=data_tuple[1].lower() ,start = _from , end = _to)
        h2 = swap_structure(dex = data_tuple[2].lower() ,structure=data_tuple[1].lower() ,start = _from , end = _to)
        h_int = h1.join(h2, how = 'inner')
        h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{data_tuple[0]}-{data_tuple[2]} {data_tuple[1]}'])
    elif asset == 'MD SWAP':
        h = get_md_swap(bank = data_tuple[0].upper(),structure=data_tuple[1].lower() ,start = _from , end = _to)
    elif asset == 'FX':
        h = get_fx_pair(pair = data_tuple[0].upper(),start = _from , end = _to)
    elif asset == 'OTHER/CIX':
        h = get_other(ticker = data_tuple[0], attribute= data_tuple[1], title = data_tuple[2], start = _from , end = _to)
    return h


In [21]:

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "Rob's Weekly":
        ccy = ['GBP', 'EUR', 'USD', 'EUR_6']
        _st = ['1y1y','1y1y/2y1y','5y5y','5y2y/7y3y','5y5y/10y10y','2s5s','2s10s','5s10s','5s30s','10s15s','10s30s']
        return {'SWAP': [(i,j) for i in ccy for j in _st]}
    elif broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                         ('USD','5y2y/7y3y'),('GBP','5y2y/7y3y'),('EUR','5y2y/7y3y'),('EUR_6','5y2y/7y3y'),
                         ('USD','5s7s10s'),('GBP','5s7s10s'),('EUR','5s7s10s'),('EUR_6','5s7s10s'),
                        ]}
        

In [22]:
dateset = None
PRESETS = pd.read_excel('Y:\Analytics and Tools\Swaps\\event_source.xlsx')

In [23]:
EVENTS = ['SUPPLY','CB MEETING','INFLATION PRINTS']
COUNTRY = {'USA':'USD','UK':'GBP', 'GERMANY': 'GER', 'FRANCE': 'FRA', 'ITALY': 'ITA'}
SUPPLY = {'USD 2Y': 'USB2YBC Index', 'USD 3Y': 'USN3YBC Index','USD 5Y': 'USB5YBC Index','USD 7Y': 'USB7YBC Index',
             'USD 10Y': 'USN10YBC Index','USD 20Y': 'USN20YBC Index','USD 30Y': 'USBD30YB Index',
             'GER 2Y': 'GB02CORA Index', 'GER 3Y': 'GBBU3YBC Index','GER 5Y': 'GB05CORA Index',
             'GER 7Y': 'GBBU7YBC Index', 'GER 10Y': 'GB10CORA Index','GER 20Y': 'GBBU20BC Index',
             'GER 30Y': 'GB30CORA Index','ITA 2Y': 'IRBA2YBC Index','ITA 3Y': 'IRBA03BC Index',
             'ITA 5Y': 'IRBA05BC Index','ITA 7Y': 'IRBA7YBC Index','ITA 10Y': 'IRBA10BC Index',
             'ITA 20Y': 'IRBA20BC Index','ITA 30Y': 'IRBA30BC Index','FRA 2Y': 'FRAB02BC Index',
             'FRA 3Y': 'FRAB03BC Index','FRA 5Y': 'FRAB05BC Index','FRA 7Y': 'FRAB7YBC Index',
             'FRA 10Y': 'FRAB10BC Index','FRA 20Y': 'FRAB20BC Index','FRA 30Y': 'FRAB30BC Index',
             'GBP 2Y': 'UKAB2YBC Index','GBP 3Y': 'UKAB3YBC Index','GBP 5Y': 'UKAB5YBC Index','GBP 7Y': 'UKAB7YBC Index',
             'GBP 10Y': 'UKAB10BC Index','GBP 20Y': 'UKAB20BC Index','GBP 30Y': 'UKAB30BC Index'}

eventW = Dropdown(options = ['-'] + EVENTS ,description = 'Event')

output3 = Output()

def build_event_list(event):
    
    
    style = {'description_width': 'initial'}
    
    cont = {'-': Text(value = 'Choose Event',disabled = True),
            'SUPPLY' : Dropdown(options = [i.upper() for i in COUNTRY.keys()], description = 'Country'),
            'CB MEETING': Dropdown(options = ['FOMC','ECB','BOE','RBA'],description = 'CB'),
           'INFLATION PRINTS': Dropdown(options = ['US CPI','Eurozone HICP','UK RPI', 'SA CPI'],description = 'Index')}

    supp = {'-': Text(value = '-',disabled = True),
            'SUPPLY' : Dropdown(options = ['2Y','3Y','5Y','7Y','10Y','20Y','30Y'], description = 'Tenor'),
            'CB MEETING': Text(value = '-',disabled = True),
           'INFLATION PRINTS': Text(value = '-',disabled = True)}
    
    year = {'-': Text(value = '-',disabled = True),
            'SUPPLY' : Dropdown(options = range(_FROM,_TO), description = 'Lookback'),
            'CB MEETING': Text(value = '-',disabled = True),
           'INFLATION PRINTS': Text(value = '-',disabled = True)}

    
    confirm_event = Button(description = 'Confirm')
    
    global dateset

    def on_button_clicked3(b):
        global dateset
        with output3:
            clear_output()
            if event =='-':
                pass
            else:
                if event == 'SUPPLY':
                    auc = SUPPLY[f"{COUNTRY[cont[event].value]} {supp[event].value}"]
                    dateset = extract_auction_dates(auc,year[event].value) 
                    print(f'Dates for {cont[event].value} {supp[event].value} supply loaded. Select structures below and then pull data')
                    
                elif event == 'CB MEETING':
                    dateset = pd.DataFrame({}, index = pd.to_datetime(PRESETS[cont[event].value].dropna()))
                    dateset = dateset.index.sort_values()
                    print(f'Dates for {cont[event].value} meetings loaded. Select structures below and then pull data')
                    
                elif event == 'INFLATION PRINTS':
                    dateset = pd.DataFrame({}, index = pd.to_datetime(PRESETS[cont[event].value].dropna()))
                    dateset = dateset.index.sort_values()
                    print(f'Dates for {cont[event].value} loaded. Select structures below and then pull data')
                
                
                
                    
               
                
                    
    display(HBox([cont[event],supp[event],year[event]]))
    display(confirm_event,output3)
        
        
    confirm_event.on_click(on_button_clicked3)
    
    
    
interactive(build_event_list,event=eventW)


interactive(children=(Dropdown(description='Event', options=('-', 'SUPPLY', 'CB MEETING', 'INFLATION PRINTS'),…

In [24]:
trades = {'SWAP':[],'FUTURE':[],'BOND':[], 'BOND FUTURE': [],'ASW':[],'MD SWAP': [] ,'XMKT SWAP':[],'FX':[] ,'OTHER/CIX': []}
master = pd.DataFrame({})


In [25]:

x = ['-'] + [i for i in trades.keys()]
assetW = Dropdown(options = x,description = 'Instrument')

output1 = Output()
output2 = Output()
outputn = Output()

def build_data_list(asset):

    eugov = ['DBR','SPGB','FRTR','BTP','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB']

    
    style = {'description_width': 'initial'}
    b_or_i = {'-': Text(value = 'Choose Instrument',disabled = True),
        'SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Currency'),
         'FUTURE':Dropdown(options = ['USD','EUR','GBP'], description = 'Currency'),
         'BOND':Dropdown(options = ['UST','DBR','SPGB','FRTR','BTP','UKT','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB'], description = 'Bond'),
         'BOND FUTURE': Dropdown(options = ['Yield', 'Price'], description = 'Field'),
        'MD SWAP': Dropdown(options = ['FOMC','ECB','MPC'], description = 'Central Bank'),
         'ASW':Dropdown(options = ['UST','DBR','SPGB','FRTR','BTP','UKT','BTP-DBR','FRTR-DBR','SPGB-DBR','BTP-SPGB','FRTR-SPGB'], description = 'Bond'),
              'FX': Text(description = 'Pair'),
              'XMKT SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Currency'),
              'OTHER/CIX':Text(description = 'Ticker')}
    
    what_trade = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Text(description = 'Structure'),
         'FUTURE':Text(description = 'Structure'),
         'BOND':Text(description = 'Structure'),
        'BOND FUTURE':Text(description = 'Structure'),
         'ASW':Text(description = 'Structure'),
        'MD SWAP': Text(description = 'Structure'),
        'XMKT SWAP':Text(description = 'Structure'),
                  'FX':Text(value = ' ',disabled = True),
        'OTHER/CIX':Dropdown(options = ['','Last Price','YTM'],description = 'Attribute')}
    
    
    what_trade_p = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Dropdown(options = ['-']+swap_ops, description = 'Preset'),
         'FUTURE':Dropdown(options = ['-']+clr_dd, description = 'Preset'),
         'BOND':Dropdown(options = ['-']+bond_ops, description = 'Preset'),
        'BOND FUTURE':Text(value = '-',disabled = True),
         'ASW':Dropdown(options = ['-']+bond_ops, description = 'Preset'),
             'MD SWAP':Text(value = '-',disabled = True),
             'XMKT SWAP':Text(value = '-',disabled = True),
             'FX': Text(value = '-',disabled = True),
             'OTHER/CIX':Dropdown(options = ['-',"Rob's Weekly","MR Presets"], description = 'Preset')}
    
    final = {'-': Text(value = 'Choose Instrument',disabled = True),
                  'SWAP':Text(value = '-',disabled = True),
         'FUTURE':Text(value = '-',disabled = True),
         'BOND':Text(value = '-',disabled = True),
        'BOND FUTURE':Text(value = '-',disabled = True),
         'ASW':Dropdown(options = ['ESTR','EURIBOR'] , description = 'Against (if EUGOV)',style=style),
             'MD SWAP':Text(value = '-',disabled = True),
             'XMKT SWAP':Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Against'),
             'FX': Text(value = '-',disabled = True),
             'OTHER/CIX':Text(description = 'Label')}
    
    include_in_list = Button(description = 'Add Structure')
    removefrom_list = Button(description = 'Remove Last')
    include__preset = Button(description = 'Add Preset')
    
    global trades
    
    
    def on_button_clicked1(b):
        with output1:
            if asset =='-' or what_trade[asset].value == '':
                pass
            else:
                if asset in ['XMKT SWAP','OTHER/CIX','ASW']:
                    trades[asset].append((b_or_i[asset].value,what_trade[asset].value,final[asset].value))
                elif asset == 'BOND FUTURE':
                    trades[asset].append((what_trade[asset].value ,b_or_i[asset].value))
                else:
                    trades[asset].append((b_or_i[asset].value,what_trade[asset].value))
                    
                assetW.value = '-'


    def on_button_clickedn(b):
        with outputn:
            if asset =='-' or what_trade_p[asset].value == '-':
                pass
            else:
                if asset == 'FUTURE':
                    for tup in fut_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'SWAP':
                    for tup in swap_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'BOND':
                    for tup in bond_spline_presets(b_or_i[asset].value,what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'ASW':
                    for tup in asw_spline_presets(b_or_i[asset].value,what_trade_p[asset].value,final[asset].value):
                        trades[asset].append(tup)
                        
                elif asset == 'OTHER/CIX':
                    custom_trades = custom_presets(what_trade_p[asset].value) 
                    for a in custom_trades.keys():
                        trades[a] += custom_trades[a]
                    
                assetW.value = '-'    
                                     
                
    def on_button_clicked2(b):
        with output2:
            if assetW == '-':
                pass
            else:
                trades[asset].pop()
                assetW.value = '-'
    
    
    
    display(HBox([b_or_i[asset],what_trade[asset],final[asset]]))
    display(what_trade_p[asset])
        
        
    include_in_list.on_click(on_button_clicked1)
    removefrom_list.on_click(on_button_clicked2)
    include__preset.on_click(on_button_clickedn)
    display(HBox([include_in_list,removefrom_list,include__preset]))
    
    max_len = max([len(trades[i]) for i in trades.keys()])
    y = {}
    for i in trades.keys():
        if i == 'XMKT SWAP':
            y[i] = [f'{i[0]}-{i[2]} {i[1]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
        elif i  =='ASW':# and i[0] in eugov:
            
            y[i] = [f'{i[0]}-{i[1]} v {i[2]}'if i[0] in eugov else f'{i[0]} {i[1]}' for i in trades[i] ] + [float('nan')]*(max_len - len(trades[i]))
        
        elif i == 'BOND FUTURE':
            y[i] = [j[0] for j in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
            
        elif i == 'OTHER/CIX':
            y[i] = [f'{i[2]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
            
        else:
            y[i] = [f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')]*(max_len - len(trades[i]))
    
    display(pd.DataFrame(y).style.format('{}',na_rep = ''))
    
interactive(build_data_list,asset=assetW)


interactive(children=(Dropdown(description='Instrument', options=('-', 'SWAP', 'FUTURE', 'BOND', 'BOND FUTURE'…

In [26]:
master = None
runs = None

In [27]:
BBG_B = Button(description = 'Pull BBG Data')
output5 = Output()

display(BBG_B,output5)

def on_button_clicked5(b):
    global master
    global runs
    with output5:
        clear_output()
        start = dateset[0] - td(365)
        master = function_for_bb(trades, start,'today')
        all_dates = pd.Series(master.index)
        
        runs  = {i:pd.DataFrame({}, columns = range(-10,11)) for i in master.columns}
        
        
        for i in dateset:
            where = all_dates[all_dates == i].index[0] 
            event_window = all_dates[where-10: where+11]
            for j in runs.keys():
                runs[j].loc[i.strftime('%b %y')] = master.loc[event_window, j].values
                #runs[j] -= runs[j][0]        
        print('Data prepared. Tools below are ready for use')

BBG_B.on_click(on_button_clicked5)

Button(description='Pull BBG Data', style=ButtonStyle())

Output()

## MOVE ON AN EVENT DAY


In [28]:
WINDOWS = {'1W':5,'2W':10,'1M':20,'3M':60,'6M':120,'1Y':250}
DATESET1 = None
z_table = None

In [29]:
LOOKBACK_W = Dropdown(options = WINDOWS.keys(), description = 'Lookback')
DATE_OF_STUDY_W = Dropdown(options = ['-'], description = 'As of')
TRADE1_W = Dropdown(options = ['-'], description = 'Trade')

output6 = Output()
output7 = Output()

ZSCORE_B = Button(description = 'Show Z-Scores')
UPDATE1_B = Button(description = 'Update Dropdown')

DIST_B = Button(description = 'Plot Density')
UPDATE2_B = Button(description = 'Update Dropdown')

display(LOOKBACK_W,DATE_OF_STUDY_W)
display(ZSCORE_B,UPDATE1_B,output6)

def on_button_clicked6(b):
    global z_table
    with output6:
        clear_output()
        if DATE_OF_STUDY_W.options == ('-',):
            print('Update the Dropdown')
        else:
            print(master)
            lb = WINDOWS[LOOKBACK_W.value]
            all_dates = pd.Series(master.index)
            asof = all_dates[all_dates == DATESET1[DATE_OF_STUDY_W.value]].index[0]
            subset = master.copy().loc[all_dates[asof - lb:asof+1]].diff()
            
            z_table = pd.DataFrame({}, index = master.columns)
             
            z_table['LEVEL'] = master.loc[master.index[asof]].values
            z_table['EVENT DAY MOVE'] = subset.loc[subset.index[-1]]
            z_table['AVERAGE MOVE'] = subset.mean().values
            z_table['DAILY VOL'] = subset.std().values
            z_table['Z-SCORE'] = (z_table['EVENT DAY MOVE'] - z_table['AVERAGE MOVE']) / z_table['DAILY VOL']
            z_table['PERCENTILE'] = [stats.norm.cdf(i) for i in z_table['Z-SCORE']]
            display(z_table.style.format({'LEVEL': '{:.2f} bps','EVENT DAY MOVE': '{:.2f} bps','AVERAGE MOVE': '{:.2f} bps',
                                         'DAILY VOL': '{:.2f} bps','Z-SCORE': '{:.2f}','PERCENTILE': '{:.1%}',}))
            
            

def on_button_clicked7(b):
    global DATESET1
    with output6:
        clear_output()
        DATESET1 = {i.strftime('%b %y'): i for i in dateset}
        DATE_OF_STUDY_W.options = DATESET1.keys()
        


ZSCORE_B.on_click(on_button_clicked6)
UPDATE1_B.on_click(on_button_clicked7)


Dropdown(description='Lookback', options=('1W', '2W', '1M', '3M', '6M', '1Y'), value='1W')

Dropdown(description='As of', options=('-',), value='-')

Button(description='Show Z-Scores', style=ButtonStyle())

Button(description='Update Dropdown', style=ButtonStyle())

Output()

In [30]:
display(TRADE1_W)

display(DIST_B,UPDATE2_B, output7)


def on_button_clicked8(b):
    with output7:
        clear_output()
        if TRADE1_W.value == '-':
            pass
        else:
            y = TRADE1_W.value
            curr, mu, sigma = z_table.at[y,'EVENT DAY MOVE'],z_table.at[y,'AVERAGE MOVE'], z_table.at[y,'DAILY VOL']
            Z = np.linspace(-4,4,1000)
            x = [mu + (sigma * z) for z in Z]
            fig,ax = plt.subplots(figsize = (15,8))
            ax.plot(x, stats.norm.pdf(x, loc = mu, scale = sigma))
            shaded_region =  (x <= curr)
            ax.fill_between(x, stats.norm.pdf(x, loc = mu, scale = sigma), where=shaded_region,alpha = 0.3)
            ax.set_ylim(bottom = 0)
            ax.set_ylabel('pdf')
            ax.set_xlabel('Move (bps)')
            ax.set_title(f'DISTRIBUTION OF MOVES - {y}')
            plt.show()
            
        
        

def on_button_clicked9(b):
    with output7:
        clear_output()
        TRADE1_W.options = z_table.index
    
    
DIST_B.on_click(on_button_clicked8)
UPDATE2_B.on_click(on_button_clicked9)

Dropdown(description='Trade', options=('-',), value='-')

Button(description='Plot Density', style=ButtonStyle())

Button(description='Update Dropdown', style=ButtonStyle())

Output()

## MOVES ACROSS MULTIPLE EVENTS

In [31]:
MONTHS_IN_YR = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

TRADE2_W = Dropdown(options = ['-'], description = 'Trade')

ZOOM_W = IntSlider(min = 0, max = 100, step =1,value = 0, description = ' Zoom (%)')

YEARS1= [ToggleButton(value = True, description =f'{i}') for i in range(_FROM,_TO+1)]
YEARS1_W =HBox(YEARS1)

MONTHS1= [Checkbox(value = True, description = i) for i in MONTHS_IN_YR]
MONTHS1_W = VBox([HBox(MONTHS1[:4]),HBox(MONTHS1[4:8]),HBox(MONTHS1[8:])])

output8 = Output()
PLOT_EVENT_B = Button(description = 'Plot Event Runs')
UPDATE3_B = Button(description = 'Update Dropdown')

display(VBox([TRADE2_W,YEARS1_W, MONTHS1_W, ZOOM_W]))
display(PLOT_EVENT_B, UPDATE3_B, output8)

def on_button_clicked10(b):
    with output8:
        clear_output()
        if TRADE2_W.options == ('-',):
            print('Please update Dropdown')
        else:
            df = runs[TRADE2_W.value].subtract(runs[TRADE2_W.value][0], axis =0)
            yr = [int(i.description) -2000 for i in YEARS1 if i.value]
            mn = [i.description for i in MONTHS1 if i.value]
            m_and_y = [f"{i} {j}" for i in mn for j in yr]
            index_subset = [i for i in df.index if i in m_and_y]
            df = df.loc[index_subset]
            df_transpose = df.copy().T
            avg = df.mean().T
            _0, _1 = df.values.flatten().min(), df.values.flatten().max()
            
            zoom = ZOOM_W.value/100
            
            
            fig, ax = plt.subplots(figsize = (15,8))
            for i in df_transpose.columns:
                ax.plot(df_transpose[i], label = i, linestyle ='--',linewidth = 0.9)
                
                
            ax.plot(avg, label = 'Average', linewidth = 3, color = 'k')
            ax.legend()
            
            ax.set_title(f"{TRADE2_W.value} PRICE ACTION")
            ax.set_xlabel('Days relative to event day')
            ax.set_ylabel('Move')
            new_xticks = np.arange(-10, 11)
            ax.set_xticks(new_xticks)
            ax.set_xlim(-10,13)
            ax.set_ylim(_0 + zoom*abs(_0),_1 - zoom*abs(_1))
            plt.show()
            
    
def on_button_clicked11(b):
    with output8:
        clear_output()
        TRADE2_W.options = runs.keys()
    
PLOT_EVENT_B.on_click(on_button_clicked10)
UPDATE3_B.on_click(on_button_clicked11)

Button(description='Plot Event Runs', style=ButtonStyle())

Button(description='Update Dropdown', style=ButtonStyle())

Output()

## BUILD A TRADING STRATEGY

In [32]:
TRADE3_W = Dropdown(options = ['-'], description = 'Trade')

DIR_W = Dropdown(options = ['PAY','REC'], description = 'Direction')
REV_W = Checkbox(value = False, description = 'Include Reversal?')
START_W = Dropdown(options = range(-10,10), value = -10, description = 'Start Date')
REVD_W = Dropdown(options = range(-9,10), value = 0, description =  'Reverse Date')
END_W = Dropdown(options = range(-9,11), value = 10, description = 'End Date')

TRADESTR = VBox([HBox([TRADE3_W,DIR_W,REV_W]), HBox([START_W,REVD_W,END_W])])

YEARS2= [ToggleButton(value = True, description =f'{i}') for i in range(_FROM,_TO+1)]
YEARS2_W =HBox(YEARS2)

MONTHS2= [Checkbox(value = True, description = i) for i in MONTHS_IN_YR]
MONTHS2_W = VBox([HBox(MONTHS2[:4]),HBox(MONTHS2[4:8]),HBox(MONTHS2[8:])])

output9 = Output()
RESULT_B = Button(description = 'Show Result')
UPDATE4_B = Button(description = 'Update Dropdown')

display(VBox([TRADESTR,YEARS2_W, MONTHS2_W]))
display(RESULT_B,UPDATE4_B, output9)
def on_button_clicked12(b):
    with output9:
        clear_output()
        if TRADE3_W.options == ('-',):
            print('Please update Dropdown')
        else:
            df = runs[TRADE3_W.value]
            yr = [int(i.description) -2000 for i in YEARS2 if i.value]
            mn = [i.description for i in MONTHS2 if i.value]
            m_and_y = [f"{i} {j}" for i in mn for j in yr]
            index_subset = [i for i in df.index if i in m_and_y]
            df = df.loc[index_subset]
            
            sense_check1 = START_W.value < REVD_W.value < END_W.value if REV_W.value else START_W.value < END_W.value  
            
            if not sense_check1:
                print('Please check dates and/or reversal')
                
            else:
                direction = 1 if DIR_W.value == 'PAY' else -1
                day1 = df[START_W.value]
                day2 = df[REVD_W.value] if REV_W.value else df[END_W.value] 
                day3 = df[END_W.value]

                pnl = direction * ((day2 - day1) + (day2 - day3))
                
                summary = pd.DataFrame({'Trades': f'{int(pnl.count())}',
                       'Profitable' : f'{int(pnl[pnl>0].count())}',
                        'Success %':  f'{round(100 * pnl[pnl>0].count()/pnl.count(),2)}%',
                        'Sum P/L': f'{round(pnl.sum(),2)} bps',
                        'Average P/L': f'{round(pnl.mean(),2)} bps',
                        'Vol of P/L': f'{round(pnl.std(),2)} bps',
                        'Sharpe Ratio': f'{round(pnl.mean()/pnl.std(),2)}',
                        'Max':f'{round(pnl.max(),2)} bps',
                        'Min':f'{round(pnl.min(),2)} bps'  
                       }, index = [f'SUMMARY: {TRADE3_W.value}']).T
                    
                display(summary)

def on_button_clicked13(b):
     with output9:
        clear_output()
        TRADE3_W.options = runs.keys()

RESULT_B.on_click(on_button_clicked12)
UPDATE4_B.on_click(on_button_clicked13)

Button(description='Show Result', style=ButtonStyle())

Button(description='Update Dropdown', style=ButtonStyle())

Output()